In [1]:
!pip install docker


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install clickhouse_driver
import json


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

docker: Error response from daemon: Conflict. The container name "/clickhouse-server" is already in use by container "ef6dfe172af1a0247c80c820cc7ac6c3942116a38e86e2f5d5d752b1e683977d". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [4]:
from clickhouse_driver import Client

client = Client(host='localhost',  # Use 'localhost' or '127.0.0.1' for a local server
                user='default',    # Default user, adjust if you've changed the user
                password='',       # Default installation has no password for 'default' user
                port=9000)         # Default TCP port for ClickHouse

# Attempt to execute a query
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

Encountered an error: Code: 102. Unexpected packet from server localhost:9000 (expected Hello or Exception, got Unknown packet)


In [6]:
client.execute('SHOW TABLES')

[('trips',)]

In [19]:
client.execute('SHOW TABLES')

[('books',), ('trips',)]

In [21]:
# Чтение JSON файла
try:
    with open('books.json', 'r', encoding='utf-8') as file:
        books_data = json.load(file)
except FileNotFoundError:
    print("Файл books.json не найден")
    exit()
except json.JSONDecodeError:
    print("Ошибка декодирования JSON")
    exit()


# Создание таблицы
client.execute('''
CREATE TABLE IF NOT EXISTS books (
    name String,
    price Float32,
    stock UInt32,
    description String,
    created_at DateTime DEFAULT now()
) ENGINE = MergeTree()
ORDER BY (name, created_at)
''')

# Подготовка данных для вставки
insert_data = [
    (
        str(book.get('name', '')).strip(),
        float(book.get('price', 0.0)), 
        int(book.get('stock', 0)), 
        str(book.get('description', '')).strip()
    ) for book in books_data 
    if book.get('name') and book.get('price') is not None
]

# Вставка данных
client.execute('INSERT INTO books (name, price, stock, description) VALUES', insert_data)

# Проверка количества записей
result = client.execute('SELECT count() FROM books')
print(f"Inserted {result[0][0]} records")

Inserted 2000 records


In [22]:
client.execute('SHOW TABLES')  # Покажет список всех таблиц

[('books',), ('trips',)]

In [23]:
client.execute('DESCRIBE TABLE books')

[('name', 'String', '', '', '', '', ''),
 ('price', 'Float32', '', '', '', '', ''),
 ('stock', 'UInt32', '', '', '', '', ''),
 ('description', 'String', '', '', '', '', '')]

In [24]:
import pandas as pd

In [30]:
result = client.execute('SELECT * FROM books')
df = pd.DataFrame(result, columns=['name', 'price', 'stock', 'description'])

In [31]:
df.head()

,name,price,stock,description
0,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",44.480000,8,Thomas Jefferson is often portrayed as a hopel...
1,#GIRLBOSS,50.959999,3,The founder of Nasty Gal offers a sassy and ir...
2,#HigherSelfie: Wake Up Your Life. Free Your So...,23.110001,17,There is a cosmic alarm clock going off around...
3,'Salem's Lot,49.560001,4,Thousands of miles away from the small townshi...
4,(Un)Qualified: How God Uses Broken People to D...,54.000000,16,Who You Think You Are is Not as Important as W...
